In [1]:
import sys
print(sys.version)

3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]


In [2]:
!pip install Pillow


In [15]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------ --------------------------- 3.4/11.1 MB 18.3 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 19.3 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---- ----------------------------------- 4.5/41.2 MB 24.4 MB/s eta 0:00:02
   -------- ------------------------------- 9.2/41.2 MB 23.8 MB/s eta 0:00:02
   ------------- -------------------------- 13.9/41.2 MB 22.9 MB/s eta 0:00:02
   ------------------ --------------------- 18.9/41.2 MB 23.4 MB/s eta 0:00:01
   ---------------------- ----------------- 23.6/41.2 MB 23.3 MB/s eta 0:00:01
   --------------------------- ------------ 28.6

In [3]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

import os
import pandas as pd
import numpy as np



In [4]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [5]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels


In [6]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [8]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [9]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [10]:
from tqdm import tqdm


In [11]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')  # Corrected here
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features



In [12]:
train_features = extract_features(train['image'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28821/28821 [07:22<00:00, 65.10it/s]


In [13]:
test_features = extract_features(test['image'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7066/7066 [01:48<00:00, 65.08it/s]


In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [22]:
# Ensure necessary imports are present
from tensorflow import keras # Or just: import keras
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# --- Define input shape and output classes (good practice) ---
input_shape = (48, 48, 1)
output_class = 7

# --- Build the model using the recommended list-based structure with Input ---
model = Sequential([
    # Define the input shape explicitly as the first layer
    keras.Input(shape=input_shape),

    # 1st Convolutional Block
    # Note: input_shape is removed from this Conv2D layer
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    # 2nd Convolutional Block
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    # 3rd Convolutional Block
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    # 4th Convolutional Block
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    # Flattening the convolutions
    Flatten(),

    # Fully Connected Layers
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),

    # Output Layer
    Dense(output_class, activation='softmax')
])

# --- Optional: Print model summary to verify the structure ---
# model.summary()

# --- Now you can compile and train the model as before ---
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(...)

In [24]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [25]:
model.fit(x=x_train, 
          y=y_train, 
          batch_size=128, 
          epochs=100, 
          validation_data=(x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 125s 537ms/step - accuracy: 0.2390 - loss: 1.8375 - val_accuracy: 0.2583 - val_loss: 1.8077
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 129s 570ms/step - accuracy: 0.2549 - loss: 1.8016 - val_accuracy: 0.2673 - val_loss: 1.7620
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 124s 547ms/step - accuracy: 0.2718 - loss: 1.7448 - val_accuracy: 0.3474 - val_loss: 1.6450
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 118s 524ms/step - accuracy: 0.3312 - loss: 1.6507 - val_accuracy: 0.4345 - val_loss: 1.4902
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 539ms/step - accuracy: 0.3965 - loss: 1.5346 - val_accuracy: 0.4635 - val_loss: 1.3758
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 117s 516ms/step - accuracy: 0.4251 - loss: 1.4717 - val_accuracy: 0.4932 - val_loss: 1.3201
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 119s 527ms/step - accuracy: 0.4610 - loss: 1.3985 - val_accuracy: 0.5129 - val_loss: 1.2792
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 203s 900ms/step - accuracy: 

In [26]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [27]:
from keras.models import model_from_json

In [28]:
# --- Loading the model (Method 1: JSON + Weights) ---

# Import the necessary function
# Make sure this import is run (Cell [22])

# Load architecture from JSON
# Ensure this filename EXACTLY matches your saved file
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

# Load weights into the new model
# Ensure this filename EXACTLY matches your saved file
model.load_weights("emotiondetector.h5")
print("Model loaded from emotiondetector.json and emotiondetector.h5")

# Remember to compile the model if you need to evaluate or continue training,
# unless you use the simpler load_model method below.
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model loaded from emotiondetector.json and emotiondetector.h5


In [29]:
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [36]:
def preprocess_image(image_path):
    # Load image in grayscale mode
    img = load_img(image_path, color_mode='grayscale')  # Use color_mode instead of grayscale
    # Convert to numpy array
    img_array = np.array(img)
    # Reshape for model input (add batch and channel dimensions)
    img_array = img_array.reshape(1, 48, 48, 1)
    # Normalize pixel values
    return img_array / 255.0

In [41]:
# Test with an example image
test_image_path = r'C:\Users\rinku\OneDrive\Documents\facial_emotion_detection\images\test\happy\8.jpg'
print(f"Original image category: happy")
processed_image = preprocess_image(test_image_path)
prediction = model.predict(processed_image)
predicted_emotion = emotion_labels[prediction.argmax()]
print(f"Model prediction: {predicted_emotion}")

Original image category: happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Model prediction: happy


In [43]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.1 MB 7.5 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/8.1 MB 7.2 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.1 MB 7.1 MB/s eta 0:00:01
   --------------------------- ------------ 5.5/8.1 MB 6.9 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.1 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 7.1 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 15.5 MB/s eta 0:00:00


In [44]:
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:

# Load the image (using a raw string)
img = image.open(r'C:\Users\rinku\OneDrive\Documents\facial_emotion_detection\images\test\disgust\533.jpg')

# --- rest of your code ---
# ... (the code below the image loading line)

# --- rest of your code ---
# ... (the code below the image loading line)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Display the image
ax1.imshow(img)
# First approach: Extract the emotion category outside the f-string
emotion_category = r'C:\Users\rinku\OneDrive\Documents\facial_emotion_detection\images\test\disgust\533.jpg'.split('\\')[-2]

# Now 'emotion_category' should correctly hold the string 'disgust'

# --- The rest of your code in that cell ---
ax1.set_title(f"Original: {emotion_category}\nPredicted: {predicted_emotion}") # Make sure predicted_emotion is defined earlier
ax1.set_title(f"Original: {emotion_category}\nPredicted: {predicted_emotion}")
ax1.axis('off')

# Display prediction confidence for each emotion
confidences = prediction[0]
y_pos = np.arange(len(emotion_labels))

ax2.barh(y_pos, confidences, align='center')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(emotion_labels)
ax2.set_xlabel('Confidence')
ax2.set_title('Emotion Prediction Confidence')

plt.tight_layout()
plt.show()

AttributeError: 'str' object has no attribute 'open'